In [ ]:
from nicess.bifrost import Tank, Channel, BIFROST
import scipp as sc
import numpy as np

In [ ]:
obj0 = Channel.from_calibration(sc.scalar(0., unit='degree'))
obj0.to_cadquery()

In [ ]:
obj = Tank.from_calibration(angles=sc.linspace(start=-40., stop=40., num=9, dim='channel', unit='degree'))
obj_cq = obj.to_cadquery()
obj_cq.save('test_BIFROST_secondary.step')